In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
import requests

In [ ]:
def assign_points(distance, group):
    """
    Dynamically assigns points based on amenity group and distance.
    """
    if group == 1:  
        if distance <= 0.5:
            return 2.5
        elif distance <= 1:
            return 2
        elif distance <= 1.5:
            return 1.5
    elif group == 2:  
        if distance <= 0.5:
            return 2
        elif distance <= 1:
            return 1.5
        elif distance <= 2.5:
            return 1
    return 0 

In [ ]:
# Compute distance between two locations
def get_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).miles

# Compute desirability score
def calculate_site_score(site_lat, site_lon, amenities_df, undesirable_df):
    total_points = 0
    deductions = 0

    for _, row in amenities_df.iterrows():
        amenity_type = row["amenity_type"]
        amenity_lat, amenity_lon = row["latitude"], row["longitude"]
        
        if amenity_type in DESIRABLE_AMENITIES:
            distance = get_distance(site_lat, site_lon, amenity_lat, amenity_lon)
            group = DESIRABLE_AMENITIES[amenity_type]["group"]
            points = assign_points(distance, group)
            total_points += points

    for _, row in undesirable_df.iterrows():
        undesired_lat, undesired_lon = row["latitude"], row["longitude"]
        distance = get_distance(site_lat, site_lon, undesired_lat, undesired_lon)
        if distance <= 0.25:
            deductions += 2

    final_score = max(0, total_points - deductions)
    return {"total_points": total_points, "deductions": deductions, "final_score": final_score}

# Example site coordinates
site_latitude = 33.7490
site_longitude = -84.3880

# Load datasets
amenities_df = pd.read_csv("../../data/processed/desirable_amenities.csv")
undesirable_df = pd.read_csv("../../data/processed/undesirable_activities.csv")

# Compute score
score_info = calculate_site_score(site_latitude, site_longitude, amenities_df, undesirable_df)

print("Total Points (Desirable):", score_info["total_points"])
print("Deductions (Undesirable):", score_info["deductions"])
print("Final Score:", score_info["final_score"])

From undesirable_data_processing

In [ ]:
def is_food_desert(lat, lon):
    """
    The QAP references USDA Food Access data. 
    We check if this site’s census tract is flagged as a 'food desert.'
    That typically means the LILATracts_1And20 or similar = 1.
    
    We'll demonstrate using a shapefile or CSV that has polygons for each tract
    plus columns for the LILA flags.
    """
    try:
        fd_gdf = gpd.read_file(FOOD_DESERT_SHP)
        site_point = Point(lon, lat)
        for idx, row in fd_gdf.iterrows():
            if row.geometry.contains(site_point):
                # check if row says it's a food desert
                # e.g., row["LILATracts_1And20"] == 1
                # or row["LILATracts_halfAnd10"] == 1, etc. 
                # depends on QAP’s requirement
                if row.get("LILATracts_1And20", 0) == 1:
                    return True
        return False
    except:
        return False

In [ ]:
def check_grocery_store_points(lat, lon, pool_type):
    """
    Did the project get any grocery store points? 
    We'll replicate the grocery store portion from the Desirable scoring 
    to see if it qualifies for points. If not, then we can apply the food desert deduction.
    """
    # We'll specifically re-run just for 'grocery_store'
    config = DESIRABLE_AMENITIES["grocery_store"]
    dist = fetch_nearest_desirable(lat, lon, "grocery_store", config, pool_type)
    pts = compute_desirable_points(pool_type, config["group"], dist)
    return pts > 0

In [ ]:
def get_desirable_undesirable_score(lat, lon):
    start_time = time.time()

    """
    1) Determine pool
    2) Sum up desirable points (cap at 20)
    3) Subtract 2 points for each undesirable
    4) Check food desert => -2 if no grocery store points
    Return final score.
    """
    # 1) Which Pool?
    pool_type = determine_pool(lat, lon)
    
    # 2) Desirable
    desirable_pts = calculate_desirable_score(lat, lon, pool_type)
    
    # 3) Undesirable => each distinct category is -2
    # undesirable_deduction = compute_undesirable_deduction(lat, lon)
    
    # 4) Food desert => if site is in a food desert, and no grocery store points => -2
    # food_desert_penalty = 0
    # if is_food_desert(lat, lon):
    #     got_grocery_points = check_grocery_store_points(lat, lon, pool_type)
    #     if not got_grocery_points:
    #         food_desert_penalty = 2
    
    # final_score = desirable_pts - undesirable_deduction - food_desert_penalty
    print("get_desirable_undesirable_score took ", time.time() - start_time, "seconds")

    final_score = desirable_pts
    return final_score

In [ ]:
site_lat = 30.98833
site_lon = -82.89667
score = identify_undesirable_activities(site_lat, site_lon, radius_miles=0.25)
print(score)


In [ ]:
if __name__ == "__main__":
    # Example usage: pick a lat/lon in GA
    site_lat = 30.818623
    site_lon = -83.265127
    score = get_desirable_undesirable_score(site_lat, site_lon)
    print(f"Final Desirable/Undesirable Score for site at ({site_lat}, {site_lon}): {score}")